In [1]:
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import itertools
from skimage.morphology import label
from copy import deepcopy as dcopy,copy
from scipy.signal import convolve2d

In [2]:
    # valid_grid = np.array([[0, -1, -1, -1, -1],
#                        [-1, -1, 9, -1, 7],
#                        [8, -1, -1, -1, -1],
#                        [-1, -1, 15, -1, 12],
#                        [10, -1, -1, -1, -1]])
# N = 5
# blankcoords = [[0,0],[1,2],[1,4],[2,0],[3,2],[3,4],[4,0]]

valid_grid = np.array([ [-1, 18, -1, -1, -1, -1, 7, -1, -1],
                        [-1, -1, -1, -1, 12, -1, -1, -1, -1],
                        [-1, -1, 9, -1, -1, -1, -1, 31, -1],
                        [-1, -1, -1, -1, -1, -1, -1, -1, -1],
                        [-1, 5, -1, 11, -1, 22, -1, 22, -1],
                        [-1, -1, -1, -1, -1, -1, -1, -1, -1],
                        [-1, 9, -1, -1, -1, -1, 19, -1, -1],
                        [-1, -1, -1, -1, 14, -1, -1, -1, -1],
                        [-1, -1, 22, -1, -1, -1, -1, 15, -1]])

N = 9

blankcoords = [[0,1],[0,6],[1,4],[2,2],[2,7],[4,1],[4,3],[4,5],[4,7],[6,1],[6,6],[7,4],[8,2],[8,7]]

blankcoords_dict = {0:[[0,1],[0,6]],1: [[1,4]],2 : [[2,2],[2,7]],3:[],4 : [[4,1],[4,3],[4,5],[4,7]],5:[], 6 : [[6,1],[6,6]], 7 : [[7,4]], 8 : [[8,2],[8,7]]}

splits = np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)

In [3]:
def check_blankcoords(grid):
    for coord in blankcoords:
        coord_r = coord[0];coord_c = coord[1]
        surr_coords = [[coord_r - 1,coord_c],[coord_r + 1,coord_c],[coord_r,coord_c - 1],[coord_r,coord_c + 1]]
        surr_vals = []
        for surr_coord in surr_coords:
            if((surr_coord[0] >= 0) and (surr_coord[1] >= 0) and (surr_coord[0] < N) and (surr_coord[1] < N)):
                surr_vals.append(grid[surr_coord[0],surr_coord[1]])
        if -1 in surr_vals:
            continue
        #print(valid_grid[coord_r,coord_c])
        sums = set()
        for r in range(0, len(surr_vals) + 1):
            for combo in itertools.combinations(surr_vals, r):
                sums.add(sum(combo))
        #print(sums)
        if valid_grid[coord_r,coord_c] not in sums:
            return False
    return True

In [4]:
class Grid():
    def __init__(self):
        self.potential_grids = [[N,np.ones((N,N),dtype=int)*-1,[0,0,N,N],list(range(2,N+1)),[]]] 
        self.splits =  np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)
     
    def add_layer(self,grid,rangecoords,lvl,alignment,coords):
        row_start,col_start,row_end, col_end = rangecoords
        new_grid = dcopy(grid)
        new_coords = dcopy(coords)

        if alignment == 0:
            new_grid[row_start:row_end,col_start] =lvl
            new_grid[row_start,col_start:col_end] =lvl
            
            # if(not self.check([row_start,col_start],new_grid,valid_grid)):
            #     return -1,-1,-1,-1
            new_coords.append([row_start,col_start])
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            new_grid[row_start:row_end,col_start] =lvl
            new_grid[row_end-1,col_start:col_end] =lvl
            
            # if(not self.check([row_end-1,col_start],new_grid,valid_grid)):
            #     return -1,-1,-1,-1
            new_coords.append([row_end-1,col_start])
            row_end -=1
            col_start +=1


        if alignment == 2:
            new_grid[row_start:row_end,col_end-1] =lvl
            new_grid[row_start,col_start:col_end] =lvl
            
            # if(not self.check([row_start,col_end-1],new_grid,valid_grid)):
            #     return -1,-1,-1,-1
            new_coords.append([row_start,col_end-1])
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            new_grid[row_start:row_end,col_end-1] =lvl
            new_grid[row_end-1,col_start:col_end] =lvl
            
            # if(not self.check([row_end-1,col_end-1],new_grid,valid_grid)):
            #     return -1,-1,-1,-1
            new_coords.append([row_end-1,col_end-1])
            row_end -=1
            col_end -=1
    
        new_rangecoords = [row_start,col_start,row_end,col_end]


        return 1,new_grid,new_rangecoords,new_coords
    
    def solve(self):
        solutions = []
        while len(self.potential_grids) > 0:
            l,g,rc,u,c = self.potential_grids.pop(0)
            lvl = dcopy(l)
            grid = dcopy(g)
            rangecoords = dcopy(rc)
            unused = dcopy(u)
            coords = dcopy(c)
            if lvl == 0:
                if check_blankcoords(grid):
                    solutions.append([grid,coords])
            else:
                #print(grid)
                if lvl == 1:
                    num = 1
                    b,new_grid,new_rangecoords,new_coords = self.add_layer(grid,rangecoords,num,0,coords)
                    if b == 1:
                        self.potential_grids.append([lvl-1,new_grid,new_rangecoords,[],new_coords])
                else:
                    for num in unused:
                        if num < 2*lvl:
                            for alignment in range(4):
                                b,new_grid,new_rangecoords,new_coords = self.add_layer(grid,rangecoords,num,alignment,coords)
                                if b == 1 and check_blankcoords(new_grid):
                                    self.potential_grids.append([lvl-1,new_grid,new_rangecoords,[n for n in unused if n != num],new_coords])
        return solutions
    


In [5]:
test = Grid()
soln_list = test.solve()

In [6]:
grid = soln_list[0][0].copy()
grid

array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
       [7, 7, 7, 7, 7, 7, 7, 7, 9],
       [5, 5, 5, 5, 5, 5, 8, 7, 9],
       [5, 4, 4, 4, 4, 4, 8, 7, 9],
       [5, 2, 2, 2, 6, 4, 8, 7, 9],
       [5, 2, 3, 3, 6, 4, 8, 7, 9],
       [5, 2, 3, 1, 6, 4, 8, 7, 9],
       [5, 6, 6, 6, 6, 4, 8, 7, 9],
       [8, 8, 8, 8, 8, 8, 8, 7, 9]])

In [14]:
ones_grid = np.ones((N,N),dtype=int)
splits = np.array([list(i) for i in itertools.product([0, 1], repeat = N)],dtype=int)

def twobytwo(grid):
    return np.any(convolve2d(grid > 0, np.ones((2, 2)), mode='valid') == 4)

def surroundingvals(grid,coord):
    lis = [[coord[0],coord[1] - 1],[coord[0],coord[1] + 1],[coord[0] + 1,coord[1]],[coord[0] - 1,coord[1]]]
    vals = []
    for point in lis:
        if point[0] >= 0 and point[1] >= 0 and point[0] < 9 and point[1] < 9:
            vals.append(grid[point[0],point[1]])
    return(vals)

def check_line(ind,sample_grid,grid,split):
    grid1 = sample_grid.copy()
    grid1[ind,:] = split*grid[ind,:]
    if np.max(label(grid1!=0,connectivity=1)) != 1:
        return False,0
    
    if twobytwo(grid1):
        return False,0
    #valid_line = valid_grid[ind]
    #print(grid)
    #print(ind)
    for coord in blankcoords_dict[ind]:
        #print(coord)
        if grid1[coord[0],coord[1]] != 0:
            return False,0
    for ind2 in range(0,ind + 1):
        for point in blankcoords_dict[ind2]:
            #print(point)
            surr_vals = surroundingvals(grid1,point)
            #print(surr_vals)
            if -1 in surr_vals:
                val_sum = sum(val for val in surr_vals if val != -1)
                #print(val_sum)
                if(val_sum > valid_grid[point[0],point[1]]):
                    return False,0
            else:
                val_sum = sum(val for val in surr_vals)        
                if(val_sum != valid_grid[point[0],point[1]]):
                    return False,0
    
    #print(grid1)
    return True,grid1



In [15]:
total_grids = []

for ind,soln_s in enumerate(soln_list):
    grid = soln_s[0]
    #print(grid)
    potential_grids = [[0,np.ones((N,N),dtype=int)*-1]]

    while len(potential_grids) > 0:
        l,gridg = potential_grids.pop(0)
        lvl = dcopy(l)
        sample_grid = dcopy(gridg)
        #print(lvl)
        if lvl == 9:
            total_grids.append(sample_grid)
        else:
            for split in splits:
                truth_value,new_grid = check_line(lvl,sample_grid,grid,split)
                if truth_value:
                    potential_grids.append([lvl + 1,new_grid])

[[9 9 9 9 9 9 9 9 9]
 [7 7 7 7 7 7 7 7 9]
 [5 5 5 5 5 5 8 7 9]
 [5 4 4 4 4 4 8 7 9]
 [5 2 2 2 6 4 8 7 9]
 [5 2 3 3 6 4 8 7 9]
 [5 2 3 1 6 4 8 7 9]
 [5 6 6 6 6 4 8 7 9]
 [8 8 8 8 8 8 8 7 9]]
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
[[9 9 9 9 9 9 9 9 9]
 [7 7 7 7 7 7 7 7 9]
 [5 5 5 5 5 5 8 7 9]
 [5 4 4 4 4 4 8 7 9]
 [5 2 2 2 6 4 8 7 9]
 [5 2 3 1 6 4 8 7 9]
 [5 2 3 3 6 4 8 7 9]
 [5 6 6 6 6 4 8 7 9]
 [8 8 8 8 8 8 8 7 9]]
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [32]:
def check_num(fin_grid):
    for ind in range(1,10):
        if (np.sum(fin_grid == ind) != ind):
            return False
    return True

for fin_grid in total_grids:
    if check_num(fin_grid):    
        print(fin_grid)
        print("\n\n")

[[9 0 9 9 0 0 0 0 0]
 [7 0 0 7 0 0 7 7 9]
 [6 0 0 5 5 5 8 0 9]
 [6 3 4 4 0 5 0 7 9]
 [0 0 2 0 4 0 8 0 0]
 [0 0 2 1 4 5 8 7 9]
 [0 0 3 0 0 3 0 0 9]
 [6 6 6 6 0 0 8 7 9]
 [0 8 0 8 8 0 8 0 0]]



